In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp_api
import kerastuner
import numpy as np
import pandas as pd
import os
import json
import datetime
import dill

In [2]:
from tensorflow.keras.layers import (
    TimeDistributed, 
    Dense, 
    Conv1D, 
    MaxPooling1D, 
    Bidirectional, 
    LSTM, 
    Dropout,
    Lambda
)

In [26]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [6]:
dataset_name = "SEG_AR"

In [7]:
static_params = {
    'PAST_HISTORY': 16,
    'FUTURE_TARGET': 8,
    'BATCH_SIZE': 1024,
    'BUFFER_SIZE': 200000,
    'EPOCHS': 500,
    'VOCAB_SIZE': 16293
 }

In [8]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [29]:
train_set = np.genfromtxt("data/SEG_train_set.csv", delimiter="\n", dtype=np.int32)
x_train, y_train = generate_timeseries(train_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().batch(static_params["BATCH_SIZE"]).shuffle(static_params["BUFFER_SIZE"])

In [30]:
val_set = np.genfromtxt("data/SEG_val_set.csv", delimiter="\n", dtype=np.int32)
x_val, y_val = generate_timeseries(val_set, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
val_data = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_data = val_data.cache().batch(static_params["BATCH_SIZE"])

In [11]:
x_train.shape

(161267, 16, 1)

In [49]:
class FeedBack(keras.Model):
    def __init__(self, units, out_steps):
        super().__init__()
        self.out_steps = out_steps
        self.units = units
        self.lstm_cell = keras.layers.LSTMCell(units)

        # Also wrap the LSTMCell in an RNN to simplify the `warmup` method.
        self.lstm_rnn = tf.keras.layers.RNN(self.lstm_cell, return_state=True)
        self.dense = Dense(static_params["VOCAB_SIZE"], activation="softmax")
        #self.decode = Lambda(lambda x: tf.reshape(tf.cast(tf.math.argmax(x, axis=1), tf.float32), (-1, 1)))

In [72]:
feedback_model = FeedBack(units=128, out_steps=static_params["FUTURE_TARGET"])

In [73]:
def warmup(self, inputs):
    onehot_inputs = tf.squeeze(tf.one_hot(inputs, static_params["VOCAB_SIZE"]), axis=2)

    # inputs.shape => (batch, time, features)
    # x.shape => (batch, lstm_units)
    x, *state = self.lstm_rnn(onehot_inputs)

    # predictions.shape => (batch, features)
    #prediction = self.dense(x)
    prediction = self.dense(x)
    #prediction = self.decode(prediction)

    return prediction, state

FeedBack.warmup = warmup

In [52]:
prediction, state = feedback_model.warmup(x_train[:2].reshape(2, -1, 1))
prediction

<tf.Tensor: shape=(2, 16293), dtype=float32, numpy=
array([[6.1300219e-05, 6.1480685e-05, 6.1337698e-05, ..., 6.1370658e-05,
        6.1381965e-05, 6.1353712e-05],
       [6.1300219e-05, 6.1480685e-05, 6.1337698e-05, ..., 6.1370658e-05,
        6.1381965e-05, 6.1353712e-05]], dtype=float32)>

In [74]:
def call(self, inputs, training=None):
    # Use a TensorArray to capture dynamically unrolled outputs.
    predictions = []
    # Initialize the lstm state
    prediction, state = self.warmup(inputs)

    # Insert the first prediction
    predictions.append(prediction)

    # Run the rest of the prediction steps
    for _ in range(self.out_steps - 1):
        # Use the last prediction as input.
        x = prediction

        # Execute one lstm step.
        x, state = self.lstm_cell(x, states=state, training=training)

        # Convert the lstm output to a prediction.
        #prediction = self.dense(x)
        prediction = self.dense(x)
        #prediction = self.decode(prediction)


        # Add the prediction to the output
        predictions.append(prediction)

    # predictions.shape => (time, batch, features)
    predictions = tf.stack(predictions)

    # predictions.shape => (batch, time, features)
    predictions = tf.transpose(predictions, [1, 0, 2])
    return predictions

FeedBack.call = call

In [75]:
result = feedback_model(x_train[:2].reshape(2, -1, 1))
result.shape

TensorShape([2, 8, 16293])

In [76]:
feedback_model.compile(
    optimizer=keras.optimizers.Nadam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

In [79]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "logs/" + timestamp

os.makedirs(log_dir)
timestamp

'20200916-191719'

In [80]:
history = feedback_model.fit(train_data, validation_data=val_data, epochs=50, callbacks=[
        keras.callbacks.EarlyStopping('val_accuracy', patience=10),
        keras.callbacks.TensorBoard(log_dir)
        ])

 ETA: 21s - loss: 1.1723 - accuracy: 0.622 81/158 [==============>...............] - ETA: 21s - loss: 1.1764 - accuracy: 0.621 82/158 [==============>...............] - ETA: 21s - loss: 1.1784 - accuracy: 0.620 83/158 [==============>...............] - ETA: 21s - loss: 1.1790 - accuracy: 0.620 84/158 [==============>...............] - ETA: 20s - loss: 1.1787 - accuracy: 0.620 85/158 [===============>..............] - ETA: 20s - loss: 1.1816 - accuracy: 0.619 86/158 [===============>..............] - ETA: 20s - loss: 1.1836 - accuracy: 0.618 87/158 [===============>..............] - ETA: 19s - loss: 1.1834 - accuracy: 0.619 88/158 [===============>..............] - ETA: 19s - loss: 1.1860 - accuracy: 0.618 89/158 [===============>..............] - ETA: 19s - loss: 1.1862 - accuracy: 0.617 90/158 [================>.............] - ETA: 19s - loss: 1.1848 - accuracy: 0.618 91/158 [================>.............] - ETA: 18s - loss: 1.1756 - accuracy: 0.621 92/158 [================>........